<a href="https://colab.research.google.com/github/MANOJ9590/SPA/blob/main/SmartProcessAnalytics(Task2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 2 - Predictive Analysis based on the event log

---
Group 12


*   List item
*   List item
*   List item
*   List item
*   List item
*   List item




In [ ]:
!pip install pm4py

In [2]:
import pm4py
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
def import_xes(filepath):
  return pm4py.read_xes(filepath)

In [5]:
def events_to_cases(eventlog):
  case={}
  complete_events = []
  for concept in range(0,len(eventlog)) :
    events = []
    for name in range(0 ,len(eventlog[concept])):
      events.append(eventlog[concept][name]['concept:name'])
      complete_events.append(eventlog[concept][name]['concept:name'])
      case[concept] = events
  return case,complete_events

In [6]:
def runing_instance_sublist_check(Ngram_sequence,trace):
    for i in range(len(trace) - len(Ngram_sequence) + 1):
      if ((Ngram_sequence) == (trace[i:i+len(Ngram_sequence)])): return True
    return False

In [7]:
def computing_probabilites(events_log, n_th_gram, running_instance):
  if (n_th_gram <= len(running_instance)):
    predections=[]
    incremental_predection_probability={}
    for i in range(n_th_gram):
      if(n_th_gram >=1):
        Ngram_seq = running_instance[-n_th_gram : ]
        n_th_element = Ngram_seq[len(Ngram_seq)-1]
        for j in range(len(events_log)):
          if(runing_instance_sublist_check(Ngram_seq,events_log[j])==True):
            next_event=(events_log[j].index(n_th_element)+1)
            if(len(events_log[j]) == next_event):
              continue
            predection = events_log[j][next_event]
            predections.append(predection)
        max_occurance = max(predections, key = predections.count)
        occurance = dict((element,predections.count(element)) for element in set(predections))
        probabilities = {}
        for k, v in occurance.items():
          probabilities[k] = round(v /sum(occurance.values()),4)
      incremental_predection_probability[str(n_th_gram) + "- Gram"] = probabilities
      n_th_gram -= 1
    return pd.DataFrame(incremental_predection_probability).fillna(0)
  else : 
    print("range of Ngram value should be less or equal to number of events in running instance input provided")

In [8]:
cases,Total_events=events_to_cases(import_xes('/content/drive/MyDrive/BPIChallenge2018.xes'))

parsing log, completed traces ::   0%|          | 0/14424 [00:00<?, ?it/s]

In [9]:
import yaml
#insert any number between 1 to 14424 to get the sequece to design the running sequence in the next step 
print(yaml.dump(cases[23]))

[mail valid, initialize, begin editing, finish pre-check, finish editing, initialize,
  begin editing, finish editing, finish editing, create, mail income, begin editing,
  begin editing, finish pre-check, finish editing, create, begin editing, begin editing,
  finish pre-check, finish editing, create, initialize, performed, initialize, begin
    editing, calculate, finish editing, begin editing, calculate, finish editing,
  begin editing, calculate, finish editing, begin editing, calculate, finish editing,
  begin editing, calculate, finish editing, begin editing, calculate, finish editing,
  begin editing, calculate, finish editing, begin editing, calculate, finish editing,
  begin editing, calculate, calculate, finish editing, decide, begin payment, insert
    document, insert document, finish payment]



In [10]:
running_process_instance = ['mail valid', 'initialize']
# enter the number less than or equal to nuber of event sin runnning instance input provided
Ngram_input = 2
print("Dataset contains",str(len(Total_events)), "events suming to", str(len(cases)) ,"cases\n",)
print('\033[1m'+"Predective analysis \n\n")
print(computing_probabilites(cases,Ngram_input,running_process_instance).to_markdown(tablefmt="grid"))

Dataset contains 847383 events suming to 14424 cases

Predective analysis 


+---------------+-----------+-----------+
|               |   2- Gram |   1- Gram |
+===============+===========+===========+
| begin editing |    0.9961 |    0.9962 |
+---------------+-----------+-----------+
| initialize    |    0.0039 |    0.0038 |
+---------------+-----------+-----------+
